# Лабораторная работа №6

## Библиотека pandas

**1**.	Таблица moscow-buildings.csv содержит информацию о годах постройки жилых домов в Москве.

1)	Загрузите базу данных moscow-buildings.csv и сохраните ее как df_build. Удалите из базы данных строки, в которых значения года постройки не указаны («н.д.»), а после превратите столбец house_year в числовой.

2)	Посмотрите на минимальное и максимальное значения года постройки. Разумны ли они? Если нет, уберите из базы данных «странные» значения года постройки. Подсчитайте количество удаленных записей.

3)	Выберите из базы данных строки, соответствующие Басманному району и сохраните их в базу basm_data.

4)	Сгруппируйте данные в базе по районам и посмотрите, сколько домов в базе относятся к каждому району.

5)	Найдите средний возраст дома (возраст – в годах, считаем от текущего года) по каждому району. Подсказка: сначала добавьте переменную house_age («возраст дома»).

6)	Найдите самый старый дом и выведите информацию о нем.

**2.**	Загрузите таблицу titanic.csv и найдите ответы на следующие вопросы. Если ответ нужно приводить в процентах, то это должно быть число в интервале от 0 до 100, округленное до двух знаков, знак процента не нужен.  

1)	Посчитайте долю выживших пассажиров.

2)	Какое количество мужчин и женщин ехало на корабле? Определите долю выживших женщин и выживших мужчин.

3)	Какую долю пассажиры первого, второго и третьего классов составляли среди всех пассажиров? Определите долю выживших пассажиров каждого класса.

4)	 Посчитайте среднее и медиану возраста пассажиров. Посчитайте среднее и медиану возраста выживших и невыживших пассажиров.

5)	Добавьте столбцы FirstNameList с именем пассажира. Обратите внимание, что встречается разный формат хранения имен, в том числе существуют отличия для мужчин и женщин. Имя может быть составное (из нескольких имен) - рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Какое самое популярное женское имя на корабле? Какое самое популярное мужское имя на корабле? 

6)	Приведите список уникальных (неповторяющихся, только один человек имеет это имя) имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена. Выведите таблицу с этими пассажирами.

7)	Приведите множество всех имен пассажиров, отдельно мужских и женских. Рассмотрите оба варианта - составное имя целиком и составное имя как отдельные имена.

8)	Приведите список имен, которые одновременно встречаются у мужчин, и у женщин. Решите задачу, если рассматривать составные имена как одно имя, и если рассматривать составные имена отдельно. Выведите таблицу пассажирами, имеющими эти имена.